<a href="https://colab.research.google.com/github/thad75/TP_ENSEA_ELEVE/blob/main/2A/Option%20IA/TP3/Creating_a_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gdown
!gdown --id 1b9XN-AD2ph6Akdy4gpWdx8rIlK6HZ-u9
!pip install pytorch-lightning
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import pytorch_lightning as pl
import numpy as np
from scipy.sparse.csgraph import laplacian 
import torch
import torch.nn as nn
from torch.nn import Module
from scipy.sparse import coo_matrix
import scipy.sparse as sparse
from sklearn import preprocessing
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn import preprocessing
import torch 

from sklearn.model_selection import train_test_split

Before beginning anything :    
* Go on the following link : https://drive.google.com/drive/folders/11jRhJ9febEg7eQ8aGx0x0P8iqLpVwCNS?usp=sharing 
* Read the README file

# Recommendation Systems

A Recommendation System is a system able to recommend an item for a user depending on either :
* the content watched by the user
* the behavior of a user on items

In this lab, we will see two different Recommendation System Algorithm.

# Goal of this lab

* Understand Recommendation Systems
* Build a Graph Recommendation System
* Build a Matrix Factorisation Recommendation System
* Use Pytorch Lightning
* Understand the importance of Embeddings in Deep Learning

# I - Dataset

For this lab, we will be using a famous dataset. We will use the Amazon Luxury Dataset. The data are loaded into a Pandas Dataframe. Let's analyze and understand the dataset.



* What are the Dataframes keys ?
* How many items are there ? How many users are there ?
* What's the span of the ratings ?
* Plot the number of Ratings that are present. Is it a Gaussian Distribution ? A Skewed Gaussian Distribution ? 
* What could it mean on the quality of the products ? 


## Applying it on a real Dataset


* Using pivot methods, transform your dataframe to a Rating Matrix.

In [ ]:
# TODO : Read the downloaded CSV file 
df = pd.read_csv(...)
# TODO : Answer the previous questions 
...

# II - Matrix Factorization


## The Theory

Matrix Factorization is an easy yet powerful recommendation algorithm. In fact, it is a simple embedding model. To remind you, an embedding is another representation of a vector, in a lower dimension space. The good thing about Matrix Factorization is its usability for either content based recommendation or collaborative filtering based recommendation. This is one of the earliest model used by Netflix. 

Matrix Factorization finds latent structure in data such as it decomposes a Feedback Matrix $A\in \mathbb{R}^{M*N}$ with M the number of users and n the number of items, into :     
* $P$ :  a user embedding matrix $\in \mathbb{R}^{M*K}$
* $Q$ : an item embedding matrix $\in \mathbb{R}^{N*K}$

The embeddings are learned such that 
$ \hat{A} = P*Q^{T} \approx A$

We define the Objective Function as 

$ \underset{P \in \mathbb{R}^{M*K},  Q\in \mathbb{R}^{N*K}} min\underset{i,j\in{obs}}\sum(A_{ij} - \hat{A{ij}})^{2} $ over $P$ and  $Q$

To update the values of the matrixes, we use the typical Gradient Descent. In fact, each element of P and Q are updated using the following equations :

$p_{ik} = p_{ik} + 2\alpha*error* q_{kj}$

$q_{kj} = q_{kj} + 2\alpha*error* p_{ik}$



* Where do the $p_{ik}$ and $q_{kj}$ update formula come from ?

Now, we are going to code the Matrix Factorization Class that will perform matrix factorization for us.

The class has 4 methods :
* the init : for attribute initialization
* the error : to compute the error
* the update : to update the elements of P and Q 
* the run : to perform the calcultation for a num_step of times

In [ ]:
class Matrix_Factorization():
  """
  This class performs Matrix Factorization for a given FeedBack Matrix


  """

  def __init__(self,A, latent_dim, steps= 5000, learning_rate = 0.0002):

      self.A = A
      self.latent_dim = latent_dim
      self.steps = steps
      self.learning_rate = learning_rate

      # TODO : Initialize the correct attributes
      # M,N are the shape of the FeedBack Matrix
      self.M, self.N  = ...

      # TODO : Initialize two random matrix P and Q. Don't forget to Transpose Q
      self.P = ...
      self.Q = ...

  def error(self, a, b):
      # TODO : Return the error : the difference between a and b
      return ...

  def update(self):
    # TODO : Perform element update using gradient descent
    # TODO : Iterate among the user
    for user in range(...):
        # TODO : Iterate among the items
        for item in range(...):
          if self.A[user][item]> 0:
            # TODO : Compute the error between the GT Rating and the Predicted Rating of user on item
            error = ....
            # TODO : Using the gradient descent formula update the values of the elements of P and Q
            for latent in range(self.latent_dim):
                self.P[user][latent] = ...
                self.Q[latent][item] = ...

  
  def run(self):   
    # TODO : Call the update method 'steps' times   
    for step in range(self.steps) : 
        ...
    return np.dot(self.P, self.Q)


### Testing it on Collaborative Filtering

We give you this R matrix. As you can see the R matrix contains some outliers (Nan values)
* How can we handle the outliers in this R matrix, what changes should you do ?
* Define a function that handles the outliers of your input matrix.

In [ ]:
# Not TODO : Define a Rating Matrix. An example of Rating Matrix is given
R = np.array([[np.NaN,5,1,0,4],
              [0,0,1,0,3],
              [0,5,0,2,np.NaN],
              [1,np.NaN,0,5,1] ])

# TODO : Define a function that handles the outliers of the R matrix. Hint nan_to_num 
def correct_outliers(R):
    return ...

# TODO : Verify that your function works
R = ...


In [ ]:
# TODO : Create an instance of the Matrix_Factorization Class
latent_dim = ...
MF = Matrix_Factorization(A = ...,
                          latent_dim=...)

In [ ]:
# TODO : Calculate the approximated Rating Matrix using the run method
mat = ...

## Testing on the Luxury Dataset

Now let's test our model on the Luxury Dataset. As you've seen, there's too much data for this Algorithm to handle. The computation time would be too much. We are going to restrict the dataset for the 50 first elements.
In order to check the capacity of the algorithm. We masked few ratings to Nan values. The goal is to check if the model would be able to recapture the original rating.

In [ ]:
# TODO : Keep the 50 first elements of the df dataframe
mf_df = df.head(...)

mf_df_train = mf_df.copy()

In [ ]:
# Here we apply some Random Nan values
random.seed(10)
for i,value in enumerate(mf_df_train['Rating']):
  if random.randint(0,5)==5:
      mf_df_train['Rating'].loc[i] = np.NaN

The mf_df_train dataframe corresponds to the mf_df dataset but with some nan values in the Rating column. 
* Create a Rating Matrix using 'UserId' as Index and 'ProductId' as Columns.
* Initialize well the values of the Rating Matrix and convert it to numpy array.
* Run Matrix Factorization on the given Matrix for a feature space of size 2

In [ ]:
mf_df_train

# TODO : Using pivot method, create a Rating Matrix , with index : UserId and Columns : ProductId
R = mf_df_train.pivot(index=..., columns=..., values=...)
# TODO : Using a function defined previously, initialize the Nan values to 0
R = ...
# TODO : Run Matrix Factorization for a latent space of dimension 2 
latent_dim = ...
MF = Matrix_Factorization(R,latent_dim)
R_pred = MF.run()

Now let's answer the following questions : 
* Will user 6 like item 9 ?
* Will user 4 like item 2 ?
* What do you think of this model ? (Is it fast ? Is it efficient ? Does it need lot of engineering ?)

As you can see, it is hard to find a real way to compute the metrics of the model. Here, we just looked if the model is able to find masked items.

# III -  Deep Learning on Graphs : Graph Neural Collaborative Filtering

What we computed previously is really hard to scale at human size. Indeed, if you take the example of all the students at ENSEA (around 500 at a moment), the MF model will take long time before assessing a result that might not be optimal. Hence, we are going to see much more efficient model that uses graph.
We will use Pytorch-Lightning to create and train the model.





## Rewriting the Dataset

We will use the same Dataset as before. But, once again we will transform it to a PyTorch Dataset, to easen the next steps. As the Dataset Letters in the UserId and ProductId columns, we will encode the columns to numbers. It is the same as in the TD :)

In [ ]:
# Because of the limitation in Google Colab, we will only use 100 000 values.
df = df.head(100000)

# TODO : Using LabelEncoder transform the columns to numerical values.
UserEncoder = ...
ItemEncoder = ...

df.UserId =  UserEncoder.fit_transform(df.UserId.values)
df.ProductId =  ItemEncoder.fit_transform(df.ProductId.values)

# TODO : Write the Dataset. The Dataset should return a dictionnary of user, movie, ratings.
class AmazonLuxury(Dataset):
  def __init__(self,df):
    self.df = df

  def __getitem__(self,idx):
    # TODO : Retrieve a row from the df Dataframe using iloc 
    row =  ...
    # TODO : Return a dictionnary {'user': , 'item':, 'rating':}. Don't forget to convert it to tensors.
    return {'user': torch.tensor(row[...], dtype=torch.long),
              'item':  torch.tensor(row[...], dtype=torch.long),
              'rating':  torch.tensor(row[...], dtype=torch.float)}


  def __len__(self):
    # TODO : Return the length of the Dataset
    return ...


# TODO : Create an instance of your Dataset and verify if it returns the correct piece of data.
dataset = AmazonLuxury(...)

## Creating the DataModule

As we created the Dataset, we need some DataLoaders to send the Data to the Model for training. Let's use Pytorch Lightning to create a DataModule.

What is a DataModule ? A Lightning DataModule is just a class that gathers all the DataLoaders used for training. It's more some code management class than anything. 

In [ ]:

class LuxuryDataModule(pl.LightningDataModule):

    def __init__(self,df ):
        super().__init__()
        self.df = df
        # TODO : Define a batch size 
        self.batch_size_train, self.batch_size_valid, self.batch_size_test = ...
        

    def setup(self, stage):
        train,test = train_test_split(df, test_size=0.2)
        train, valid = train_test_split(train, test_size=0.2)
        # We need to setup our module. We have a training set that we will be fitting in our model
        # and a testing set used to test our models prediction.
        # the stage variable corresponds to those two steps : 
        #         |fit
        # stage = <test
        #         |None  

        # First stage is 'fit' (or None)

        if stage == "fit" or stage is None:
            self.amazon_train, self.amazon_val =  AmazonLuxury(train),  AmazonLuxury(valid)

        # Second stage is 'test' 
        if stage == "test" or stage is None:
            self.amazon_test =  AmazonLuxury(test)

    def train_dataloader(self):
        # TODO : Create your Training DataLoader
        return DataLoader( ..., batch_size=self.batch_size_train,drop_last=True)

    def val_dataloader(self):
        # TODO : Create  your Validation DataLoader
        return DataLoader( ..., batch_size= self.batch_size_valid,drop_last=True)

    def test_dataloader(self):
        # TODO : Create your Testing DataLoader
        return DataLoader( ..., batch_size= self.batch_size_test,drop_last=True)

## Some Definitions


An **Adjacency Matrix** is a matrix that indicates the links between nodes. In the case of User-Item Recommendation it is simply the  User-Item Interaction matrix. We will call the Adjancency Matrix $A$

A **Diagonal Degree** Matrix is a matrix that represents the degrees for each nodes of a graph. The degree of a node is the number of edge linking a given node. A loop is counted 2 times in the degree. We will call the Diagonal Matrix $D$

A **Laplacian Graph** Matrix is a matrix **that represents** the graph and relates to useful properties for a graph. We will call the Laplacian Graph Matrix L. An interesting property is : $L = D - A$



## Quick Run on NGCF

<img src = "https://miro.medium.com/max/776/1*6cBxhbNs9acjejFvsiqXMw.png" height=400> 


During the TD, we've trained a model based on Deep Learning called NCF. The model learns to rate like user U on item I, by comparing and extracting complex features from both embeddings. NGCF adds an interesting part to this model : Graphs. 

The NGCF model has 3 main components : 
* Embedding Layer
* Multiple Propagation Layers
* Prediction Layers

In comparison to the NCF model, we add the Multiple Propagation Layers. The goal of this specific layer is to propagate messages between nodes on a User Item Graph. 





NGCF works thanks to the l-hop neighbors. A 1-hop neighborhood correspond to the nodes directly connected to the given node. A 2-hop neighborhood correspond to the node that are connected through the 1-hop neighborhood to a given node.
<img src ="https://d3i71xaburhd42.cloudfront.net/c5f5f179d80a3bf9b4f29750283a87eaca42e91b/2-Figure1-1.png"  height=200>

<img src = "https://miro.medium.com/max/1400/0*rpiKmTCpuLkX7mbG" height=300>

For example, the collaborative signal i4 => u2 => i2 => u1 can be captured using the embedding propagation process. In fact, the message from i4 is encoded in the embedding of u1 from it 3-hop neighbors

## Creating a Graph Model

A Graph is an object that contains nodes and edges. Each nodes contains informations that is important. Each edges symbolizes the link between nodes. As usual, when working with GNN, we have an $aggregate function$ and a $update function$. In the case of GNCF, the goal of a Node is to aggregate information contained in the neighboring Nodes and update the central Node given the informations. When creating a GNN, we must be careful on the message-passing architecure. Indeed, it should be able to generalize to multiple succesive layers. Propagation is done using the following formulas :    

The embedding for user $u$ after memorizing the messages from the $l$-hop neighborhood is :  

$$\mathrm{{e}}_{u}^{(l)}=\mathrm{{LeakyReLU}}\Bigl({\bf m}_{u\leftarrow u}^{(l)}+\sum_{i\in N_{u}}\mathrm{m}_{u\leftarrow i}^{(l)}\Bigr)$$





The messages $m$ being propagated are defined as follows



$$
\left\{\begin{array}{l}
\mathbf{m}_{u \leftarrow i}^{(l)}=p_{u i}\left(\mathbf{W}_{1}^{(l)} \mathbf{e}_{i}^{(l-1)}+\mathbf{W}_{2}^{(l)}\left(\mathbf{e}_{i}^{(l-1)} \odot \mathbf{e}_{u}^{(l-1)}\right)\right) \\
\mathbf{m}_{u \leftarrow u}^{(l)}=\mathbf{W}_{1}^{(l)} \mathbf{e}_{u}^{(l-1)}
\end{array}\right.
$$

with 

$\mathbf{m}_{u \leftarrow i}^{(l)}$ : the message (or information) from $i$ to $u$, from its $l-hop$ neighbors 

$\mathbf{m}_{u \leftarrow u}^{(l)}$ : the message from $u$ to $u$  from its $l-hop$ neighbors, also called the self loop.

$p_{u i}$ : the first hop norm between user u and item i 

In fact, we can rewrite this as a matrix multiplication. 

For 
$$
\mathbf{E}=[\underbrace{\mathbf{e}_{u_{1}}, \cdots, \mathbf{e}_{u_{N}}}_{\text {users embeddings }}, \underbrace{\mathbf{e}_{i_{1}}, \cdots, \mathbf{e}_{i_{M}}}_{\text {item embeddings }}] .
$$
We have :
$$
\mathrm{E}^{(l)}=\operatorname{LeakyReLU}\left((\mathcal{L}+\mathrm{I}) \mathrm{E}^{(l-1)} \mathbf{W}_{1}^{(l)}+\mathcal{L} \mathrm{E}^{(l-1)} \odot \mathrm{E}^{(l-1)} \mathbf{W}_{2}^{(l)}\right)$$

with 

$\mathcal{L}$ : the normalized Laplacian matrix of the User Item Graph

$\mathrm{E}^{(l)}$ : the Representations for user and items after l steps of message propagation

$\mathrm{R}$ : the Rating Matrix aka the User-Item Interaction Matrix
$$
\mathcal{L}=\mathrm{D}^{-\frac{1}{2}} \mathrm{AD}^{-\frac{1}{2}} \text { and } \mathrm{A}=\left[\begin{array}{cc}
0 & \mathrm{R} \\
\mathrm{R}^{\top} & 0
\end{array}\right]
$$

So after propagating for L layers, we have many representation for user u and for item i. By concatenating the embeddings, we enrich the intial embedding and control the range of propagation by adjusting to the number of layers we propagate through.
$$
\mathbf{e}_{u}^{*}=\mathbf{e}_{u}^{(0)}\|\cdots\| \mathbf{e}_{u}^{(L)}, \quad \mathbf{e}_{i}^{*}=\mathbf{e}_{i}^{(0)}\|\cdots\| \mathbf{e}_{i}^{(L)}
$$

## Enough With the Theory, Let's Practice


### The GNN

The GNN Layer will forward the LaplacianMatrix (the messages from the neighbor nodes) and a SelfLoop matrix (the message from the same node). Each $W$ is a learnable weight that is updated through each updates.

We are going to create a Graph Layer that reproduces 
$$
\mathrm{E}^{(l)}=\operatorname{LeakyReLU}\left((\mathcal{L}+\mathrm{I}) \mathrm{E}^{(l-1)} \mathbf{W}_{1}^{(l)}+\mathcal{L} \mathrm{E}^{(l-1)} \odot \mathrm{E}^{(l-1)} \mathbf{W}_{2}^{(l)}\right)$$


* What known layer multiplies an input value by a learnable weight ? Linear ? Conv ? MultiHeadAttention ?
* Create the GNN Model

In [ ]:
import torch
import torch.nn as nn
from torch import Tensor


class GNNLayer(nn.Module):
    def __init__(self, in_features,  out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.linear = nn.Linear(in_features,out_features)
        self.interActTransformation = nn.Linear(in_features,out_features)

    def forward(self,features,LaplacianMat, selfLoop):

        l1 = selfLoop+ LaplacianMat 
        l2 = LaplacianMat.cuda()
        l1 = l1.cuda()
        features = features.cuda() # E
        # TODO : Recreate the above function. Be careful between the multiplication and the term by term multiplications. Hint: have a look at matmul and mul
        # TODO : Reproduce the matrix multiplication 
        inter_features = torch.matmul(...,...)  # TODO : L*E
        inter_features = torch.mul(...,...) # TODO : LE.*E

        # TODO : Send the correct variables through the Linear models to recreate both part of the Sum 
        left_term = self.linear(torch.mm(...,...)) # TODO : (L+I)E*W
        right_term = self.interActTransformation(torch.mm(...,...)) # TODO : (LE.*E)*W
        return nn.LeakyReLU()(...)



### NGCF

Now let's get our hands dirtier. We previously created our Graph Model. Now the are going to create our NGCF Model. As creating all the class would take a really long time, we gave you a skeleton to complete.


In [ ]:

class GraphNCF(pl.LightningModule):


    def __init__(self,numberUser, numberItem, embeddingSize,df):
      super().__init__()
      self.numberUser = numberUser
      self.numberItem = numberItem
      self.df = df
      self.layers = [embeddingSize, embeddingSize//2, embeddingSize//4]
      self.embeddingSize = embeddingSize

      # TODO : Define your Graph by calling one of the existing Methods. Hint : read the definitions given before
      self.laplacian = ...
      self.selfLoop = self.getSparseEye(self.numberUser + self.numberItem )

      # TODO : Create an Embedding Layer for the Users and the Items (1)
      self.userEmbed =  nn.Embedding(...,...)
      self.itemEmbed =  nn.Embedding(...,...)
      # Not TODO : Define your Propagation Layer (2). We define this part for you, it's just a for loop where you add the GNNs into a List. Each list being a L-Hop neighbor
      self.GNNLayers = nn.ModuleList()
      for From,To in zip(self.layers[:-1],self.layers[1:]):
            self.GNNLayers.append(GNNLayer(From,To))
      # TODO : Define your Prediction Layers (2). Hint : you might need to forward some data to see what the value should be
      self.model = nn.Sequential(nn.Linear(...,...))


    def configure_optimizers(self):
        # TODO : Initialize your Optimizer and your Learning Rate (Adam, SGD... ) 
        optimizer = ...
        return optimizer

    def forward(self,user, item):        
        # TODO : Call the initial FeatureMatrix and clone it. This will be our initial message. To call the feature matrix, a method in this class is present to create it for you.
        features = ...
        finalEmbed = ... # TODO : clone the features tensor
        # TODO : Create the richer embedding by sending your features through your GNNs. Don't forget the argument of your model
        for gnn in self.GNNLayers:
            features = gnn(...,...,...)
            features = nn.ReLU()(features)
            finalEmbed = torch.cat([finalEmbed,features.clone()],dim=1)
        # Not TODO : Retrieve the correct embedding 
        userEmbed , itemEmbed = finalEmbed[user], finalEmbed[item]
        Embed = torch.cat([userEmbed, itemEmbed], dim =1)
        # TODO : Send the final Embedding through your prediction layes
        pred = ...
        return pred  
        

    def training_step(self,train_batch, batch_idx):
        # TODO : Define your training step
        # TODO : Define your Loss (MSE ? BCE ? Hinge Loss ? Focal Loss ?). Hint : Are we performing regression or classification or both ?
        criterion = ...
        # TODO : Retrieve the Data from the train_batch.
        user = train_batch[...]
        item = train_batch[...]
        rating = ...
        # TODO : Send the user and the item tensor to the model.
        pred = ...
        # TODO : Compute your Loss 
        loss = ...
        return loss  


    def validation_step(self,valid_batch, batch_idx):
        # TODO : Define your validation step. How does it differs from the training step ?
        criterion = ...
        user, item, rating = ...
        pred = ...
        loss = ...

    def test_step(self,test_batch, batch_idx):
        # TODO : Define your test step. How does it differs from the training step ?
        ...
        
    def AdjacencyMat(self):
        user, product, rating = self.df['UserId'].values,self.df['ProductId'].values,self.df['Rating'].values
        R = coo_matrix((rating, (user,product))).toarray()
        N,M = R.shape
        R_t = R.T
        H1 = np.vstack([np.zeros((N,N)), R_t])
        H2 = np.vstack([R, np.zeros((M,M))])
        A = np.hstack([H1,H2])
        del H1, H2
        sumArr = (A>0).sum(axis=1)
        diag = list(np.array(sumArr.flatten()))
        diag = np.power(diag,-0.5)
        D = np.diag(diag)
        return A

    def DiagonalMat(self,A):
        sumArr = (A>0).sum(axis=1)
        diag = list(np.array(sumArr.flatten()))
        diag = np.power(diag,-0.5)
        D = sparse.diags(diag)
        return D

    def LaplacianMat(self):
        A = self.AdjacencyMat()
        D = self.DiagonalMat(A)
        L = D*A*D
        L = coo_matrix(L)
        i = torch.LongTensor([L.row,L.col])
        data = torch.FloatTensor(L.data)
        SparseL = torch.sparse.FloatTensor(i,data)
        return SparseL

    def getSparseEye(self,num):
        i = torch.LongTensor([[k for k in range(0,num)],[j for j in range(0,num)]])
        val = torch.FloatTensor([1]*num)
        return torch.sparse.FloatTensor(i,val)

    def FeatureMat(self):
        uidx = torch.LongTensor([i for i in range(self.numberUser)]).to(self.device)
        iidx = torch.LongTensor([i for i in range(self.numberItem )]).to(self.device)
        userEmbd = self.userEmbed(uidx)
        itemEmbd = self.itemEmbed(iidx)
        features = torch.cat([userEmbd,itemEmbd],dim=0)
        return features

## The Training

Now let's train the model. We normally have everything we need :     
* a DataModule to retrieve Data for Training
* a Model with all the training, testing, validation steps, an optimizer, a loss

Now we can train. We are going to create a trainer. a Trainer is a class that will handle the training for us. For our convinence, Lightning has a built-in trainer that we can use. More information : https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html

It makes thing really easy.

In [ ]:
# TODO : Create an instance of your DataModule
# TODO : Create an instance of your Model
model = ...
dm = ...
dm.setup('fit')
# TODO : Initialize the Trainer. Train the Model for 15 epochs.
trainer = pl.Trainer(gpus=-1,max_epochs=...,strategy='dp')

# Not TODO : using the .fit method, Fit the data to the model
trainer.fit(model, dm)


## The testing

Now that we trained our model, we can test it on unseen data. We are going to see how far our predictions are from the reality
* Switch the DataModule to the test stage
* Test the model.

In [ ]:
# TODO : setup the DataModule to the 'test' stage
dm.setup(...)

# TODO : Test 
trainer.test(datamodule = ...)

* Is the model efficient ?
* What could we do to have better results ?

# IV - Do it again (Optional)

In the previous part, we created a model able to regress the rating a user could give to an object. We made some rating predictions. 

Now we will change the model to know whether a user would click on an object or not.

* Change the previous model so that the model infers whether a user would click or not on a given object.

Hint :
* What task is it ?
* What should you change or add ?